#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=200,chunk_overlap=30)
docs=text_splitter.split_documents(documents)
embeddings=OllamaEmbeddings(model="llama3")
db=FAISS.from_documents(docs,embeddings)
db




Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


In [6]:
## querying
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers


In [7]:
retriever=db.as_retriever()
retriever.invoke(query)

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [9]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  19983.46),
 (Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  21353.996),
 (Document(metadata={'source

In [10]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-1.884474515914917,
 -4.870710372924805,
 4.268099784851074,
 -0.8922393321990967,
 -1.3337748050689697,
 -2.9344513416290283,
 1.6877131462097168,
 0.716935396194458,
 -2.37247896194458,
 2.749624729156494,
 -0.9059752225875854,
 1.4483070373535156,
 -1.8972837924957275,
 0.7293466925621033,
 -1.9149448871612549,
 0.573527991771698,
 -3.890197277069092,
 0.6533535718917847,
 -2.3096060752868652,
 1.7320315837860107,
 -1.9716566801071167,
 -0.5701737403869629,
 1.442807674407959,
 0.398751437664032,
 -0.28677862882614136,
 1.1677064895629883,
 0.40918296575546265,
 -1.0085725784301758,
 1.7250452041625977,
 0.039318185299634933,
 1.0114490985870361,
 0.8381984829902649,
 2.0768444538116455,
 1.7171846628189087,
 4.3673505783081055,
 -1.138685941696167,
 -0.4763711094856262,
 1.4079618453979492,
 4.454891204833984,
 3.178476095199585,
 3.679734230041504,
 -0.8826668858528137,
 -0.927720844745636,
 -0.20245389640331268,
 -0.08278411626815796,
 -0.4904390573501587,
 0.7609964609146118,
 

In [11]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [12]:
### Saving And Loading
db.save_local("faiss_index")

In [13]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [14]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content